In [1]:
## Load libraries
import os
from dotenv import load_dotenv
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# LangChain core imports
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage

# LangChain specific imports
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# Load environment variable
load_dotenv()

False

In [2]:
import json
from pathlib import Path

def load_arxiv_chunks(chunked_path, max_files=15, chunks_per_file=10):
    """Load chunked ArXiv papers from data-ingestion pipeline (limited for testing)"""
    documents = []
    json_files = sorted(chunked_path.glob("*.json"))[:max_files]
    
    print(f"Loading from {len(json_files)} ArXiv papers (max {chunks_per_file} chunks each)...")
    
    for json_file in json_files:
        with open(json_file, 'r') as f:
            chunks = json.load(f)
            for chunk in chunks[:chunks_per_file]:
                documents.append({
                    'content': chunk['content'],
                    'metadata': chunk['metadata']
                })
    
    return documents

# Path to hybrid chunked data from data-ingestion pipeline
chunked_data_path = Path("../../data-ingestion/processed/chunked/hybrid/")
arxiv_chunks = load_arxiv_chunks(chunked_data_path, max_files=15, chunks_per_file=10)

print(f"\n✅ Loaded {len(arxiv_chunks)} chunks from ArXiv papers")
print(f"\nSample ArXiv paper:")
print(f"   Title: {arxiv_chunks[0]['metadata']['title']}")
print(f"   ArXiv ID: {arxiv_chunks[0]['metadata']['arxiv_id']}")
print(f"   Category: {arxiv_chunks[0]['metadata']['primary_category']}")
print(f"   Section: {arxiv_chunks[0]['metadata']['section']}")

# Convert to LangChain Documents
sample_documents = []
for chunk in arxiv_chunks:
    doc = Document(
        page_content=chunk['content'],
        metadata={
            'arxiv_id': chunk['metadata']['arxiv_id'],
            'title': chunk['metadata']['title'],
            'category': chunk['metadata']['primary_category'],
            'section': chunk['metadata']['section'],
            'chunk_id': chunk['metadata']['chunk_id'],
            'source': 'arxiv'
        }
    )
    sample_documents.append(doc)

print(f"✅ Created {len(sample_documents)} LangChain documents")

Loading from 15 ArXiv papers (max 10 chunks each)...

✅ Loaded 150 chunks from ArXiv papers

Sample ArXiv paper:
   Title: Amortized Inference of Neuron Parameters on Analog Neuromorphic Hardware
   ArXiv ID: 2602.10763v2
   Category: cs.NE
   Section: Abstract
✅ Created 150 LangChain documents


In [4]:
# ArXiv chunks are already optimally chunked from data-ingestion
chunks = sample_documents
print(f"✅ Using {len(chunks)} pre-chunked ArXiv documents")
print(f"\nExample chunk:")
print(f"Title: {chunks[0].metadata['title'][:60]}...")
print(f"Content: {chunks[0].page_content[:150]}...")
print(f"Metadata: {chunks[0].metadata}")

✅ Using 150 pre-chunked ArXiv documents

Example chunk:
Title: Amortized Inference of Neuron Parameters on Analog Neuromorp...
Content: —Our work utilized a non-sequential simulation-based
inference algorithm to provide an amortized neural density
estimator, which approximates the post...
Metadata: {'arxiv_id': '2602.10763v2', 'title': 'Amortized Inference of Neuron Parameters on Analog Neuromorphic Hardware', 'category': 'cs.NE', 'section': 'Abstract', 'chunk_id': 0, 'source': 'arxiv'}


In [ ]:
# Initialize HuggingFace embeddings - Free, runs locally!
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


✅ HuggingFace Embeddings Loaded!
   Model: sentence-transformers/all-MiniLM-L6-v2
   Embedding Dimension: 384


In [6]:
### Compare Embedding using cosine similarity

def compare_embeddings(text1: str, text2: str):
    """Compare semantic similarity of 2 texts using embeddings"""
    emb1 = np.array(embeddings.embed_query(text1))
    emb2 = np.array(embeddings.embed_query(text2))
    similarity = np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
    return similarity

# Test semantic similarity
print("Semantic Similarity Examples:")
print(f"'AI' vs 'Artificial Intelligence': {compare_embeddings('AI', 'Artificial Intelligence'):.3f}")
print(f"'AI' vs 'Cooking recipes': {compare_embeddings('AI', 'Cooking recipes'):.3f}")

Semantic Similarity Examples:
'AI' vs 'Artificial Intelligence': 0.791
'AI' vs 'Cooking recipes': 0.152


In [7]:
### Create ChromaDB Vector Store
persist_directory = "./data/chroma_db"

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=persist_directory,
    collection_name="arxiv_papers"
)

print(f"✅ ChromaDB vector store created with {vectorstore._collection.count()} vectors")
print(f"   Persisted to: {persist_directory}")

✅ ChromaDB vector store created with 150 vectors
   Persisted to: ./data/chroma_db


In [8]:
## Load vector store from disk
loaded_vectorstore = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings,
    collection_name="arxiv_papers"
)

print(f"✅ Loaded vector store contains {loaded_vectorstore._collection.count()} vectors")

✅ Loaded vector store contains 150 vectors


/tmp/ipykernel_68462/3628335856.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  loaded_vectorstore = Chroma(


In [ ]:
## Similarity Search on ArXiv Papers
query = "What are neural networks and deep learning architectures?"

results = vectorstore.similarity_search(query, k=3)

print(f"Query: {query}\n")
print("Top 3 ArXiv papers:\n")
for i, doc in enumerate(results, 1):
    print(f"{i}. {doc.metadata['title'][:65]}...")
    print(f"   ArXiv: {doc.metadata['arxiv_id']} | Section: {doc.metadata['section']}")
    print(f"   Content: {doc.page_content[:150]}...")
    print()

🔍 Query: What are neural networks and deep learning architectures?

Top 3 ArXiv papers:

1. CL API: Real-Time Closed-Loop Interactions with Biological Neural...
   ArXiv: 2602.11632v1 | Section: methods
   Content: network (BNN) that can be harnessed directly in controllable ways as exemplified in the fields of
Synthetic Biological Intelligence (SBI) [1–3] and Or...

2. Calibrated Bayesian Deep Learning for Explainable Decision Suppor...
   ArXiv: 2602.11973v1 | Section: 1
Introduction
   Content: Deep learning has achieved remarkable advances in medical image analysis, demonstrating classification performance
comparable to that of human experts...

3. CL API: Real-Time Closed-Loop Interactions with Biological Neural...
   ArXiv: 2602.11632v1 | Section: Abstract
   Content: Biological neural networks (BNNs) are increasingly explored for their rich dy-
namics, parallelism, and adaptive behavior. Beyond understanding their ...



In [10]:
### Similarity Search with Scores
query2 = "machine learning algorithms and optimization techniques"
results_with_scores = vectorstore.similarity_search_with_score(query2, k=3)

print(f"Query: {query2}\n")
print("Results with similarity scores:\n")
for i, (doc, score) in enumerate(results_with_scores, 1):
    print(f"{i}. Similarity: {score:.4f}")
    print(f"   Paper: {doc.metadata['title'][:60]}...")
    print(f"   ArXiv: {doc.metadata['arxiv_id']} | Category: {doc.metadata['category']}")
    print(f"   Content: {doc.page_content[:120]}...")
    print()

Query: machine learning algorithms and optimization techniques

Results with similarity scores:

1. Similarity: 1.2782
   Paper: Mixed-Integer Programming for Change-point Detection...
   ArXiv: 2602.11947v1 | Category: math.OC
   Content: signal of the data, and the term breakpoint to denote the corresponding decision variable in a PWL
model at which model ...

2. Similarity: 1.2935
   Paper: Evolution With Purpose: Hierarchy-Informed Optimization of W...
   ArXiv: 2602.11398v1 | Category: cs.NE
   Content: . While
all heterogeneous strategies fit the data well, only curricular ap-
proaches generalized to new subjects. Most i...

3. Similarity: 1.3441
   Paper: Amortized Inference of Neuron Parameters on Analog Neuromorp...
   ArXiv: 2602.10763v2 | Category: cs.NE
   Content: include
directly deriving model parameters by fitting experimental
observations, genetic algorithms, gradient-based...



In [11]:
### Search with Metadata Filtering - ChromaDB supports native filtering!
query3 = "computational methods and algorithms"

# ChromaDB supports where filters on metadata
filtered_results = vectorstore.similarity_search(
    query3,
    k=3,
    filter={"source": "arxiv"}
)

print(f"🔍 Query: {query3} (filtered: source=arxiv)\n")
for i, doc in enumerate(filtered_results, 1):
    print(f"{i}. {doc.metadata['title'][:65]}...")
    print(f"   Category: {doc.metadata['category']} | Section: {doc.metadata['section']}")
    print(f"   Preview: {doc.page_content[:100]}...")
    print()

🔍 Query: computational methods and algorithms (filtered: source=arxiv)

1. Amortized Inference of Neuron Parameters on Analog Neuromorphic H...
   Category: cs.NE | Section: INTRODUCTION
   Preview: Identifying suitable models to replicate physical observations
is often a laborious process. Even wh...

2. Amortized Inference of Neuron Parameters on Analog Neuromorphic H...
   Category: cs.NE | Section: methods
   Preview: include
directly deriving model parameters by fitting experimental
observations, genetic algorithms,...

3. Mixed-Integer Programming for Change-point Detection...
   Category: math.OC | Section: 1
Introduction
   Preview: segment each data point belongs to. Therefore, by explicitly encoding segment assignments, MIP
formu...



In [ ]:
## LLM - HuggingFace Model (Chat-based)
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

# Set HuggingFace API token from .env
huggingface_token = os.getenv("HUGGINGFACE_API_KEY")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = huggingface_token

# Initialize HuggingFace LLM via chat endpoint
endpoint = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    max_new_tokens=512,
    temperature=0.7,
    huggingfacehub_api_token=huggingface_token,
)

llm = ChatHuggingFace(llm=endpoint)

print("✅ HuggingFace LLM Loaded!")
print(f"   Model: HuggingFaceH4/zephyr-7b-beta")

✅ HuggingFace LLM Loaded!
   Model: HuggingFaceH4/zephyr-7b-beta


In [13]:
# Test the LLM
response = llm.invoke("What is AI?")
print("LLM Test Response:")
print(response.content)

LLM Test Response:
AI, or artificial intelligence, refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning, reasoning, and decision making. These systems can analyze large amounts of data, identify patterns, and make informed decisions based on that analysis without being explicitly programmed to do so. AI systems can also adapt and learn over time as they interact with their environments, improving their performance through experience. AI technologies include natural language processing, computer vision, and machine learning, which enable machines to recognize patterns and objects, understand and respond to human language, and make predictions or decisions based on that understanding. AI is quickly advancing in areas such as healthcare, finance, logistics, transportation, and manufacturing, where it can help to automate processes, optimize efficiency, and improve decision-making. While AI holds tremendous promise

In [14]:
# Simple RAG Chain with LCEL
simple_prompt = ChatPromptTemplate.from_template("""Answer the question based only on the following context:
Context: {context}

Question: {question}

Answer:""")

In [15]:
## Basic retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

In [16]:
from typing import List

# Format documents for the prompt
def format_docs(docs: List[Document]) -> str:
    """Format documents for insertion into prompt"""
    formatted = []
    for i, doc in enumerate(docs):
        source = doc.metadata.get('source', 'Unknown')
        formatted.append(f"Document {i+1} (Source: {source}):\n{doc.page_content}")
    return "\n\n".join(formatted)

In [17]:
simple_rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | simple_prompt
    | llm
    | StrOutputParser()
)

In [18]:
### Conversational RAG Chain

conversational_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the provided context to answer questions."),
    ("placeholder", "{chat_history}"),
    ("human", "Context: {context}\n\nQuestion: {input}"),
])

In [19]:
def create_conversational_rag():
    """Create a conversational RAG chain with memory"""
    return (
        RunnablePassthrough.assign(
            context=lambda x: format_docs(retriever.invoke(x["input"]))
        )
        | conversational_prompt
        | llm
        | StrOutputParser()
    )

conversational_rag = create_conversational_rag()

In [20]:
### Streaming RAG chain
streaming_rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | simple_prompt
    | llm
    | StrOutputParser()
)

print("RAG chains created successfully!")
print("Available chains:")
print("- simple_rag_chain: Basic Q&A")
print("- conversational_rag: Maintains conversation history")
print("- streaming_rag_chain: Supports token streaming")

RAG chains created successfully!
Available chains:
- simple_rag_chain: Basic Q&A
- conversational_rag: Maintains conversation history
- streaming_rag_chain: Supports token streaming


In [21]:
# Test function for different chain types
def test_rag_chains(question: str):
    """Test all RAG chain variants"""
    print(f"Question: {question}")
    print("=" * 80)
    
    # 1. Simple RAG
    print("\n1. Simple RAG Chain:")
    answer = simple_rag_chain.invoke(question)
    print(f"Answer: {answer}")

    # 2. Streaming RAG
    print("\n2. Streaming RAG:")
    print("Answer: ", end="", flush=True)
    for chunk in streaming_rag_chain.stream(question):
        print(chunk, end="", flush=True)
    print()

In [22]:
test_rag_chains("What is the difference between AI and machine learning")

Question: What is the difference between AI and machine learning

1. Simple RAG Chain:
Answer: Based on the provided context, the difference between AI and machine learning is not directly addressed. Both terms are used independently in the given documents without any comparison or contrast being drawn between them. The first document discusses the use of AI for detecting fine-grained activities in newborn resuscitation videos, highlighting the challenges involved, while the second document proposes a generative AI framework based on Bayesian Neural Networks (BNNs) for a more generalized training objective. The third document introduces BNNs as a type of machine learning algorithm with biological neuron-inspired properties, such as high energy efficiency and the ability for synaptic plasticity, which is distinct from the traditional von Neumann architecture used in AI systems. Therefore, it seems that AI and machine learning are used interchangeably in this context.

2. Streaming RAG:


In [23]:
# Test with multiple questions
test_questions = [
    "What is the difference between AI and Machine Learning?",
    "Explain deep learning in simple terms",
    "How does NLP work?"
]

for question in test_questions:
    print("\n" + "=" * 80 + "\n")
    test_rag_chains(question)



Question: What is the difference between AI and Machine Learning?

1. Simple RAG Chain:
Answer: Based solely on the provided context, the given question asks about the difference between artificial intelligence (AI) and generative AI (GenAI). While both are mentioned in the first document, the second and third documents introduce alternative approaches to learning and computation: biological neurons (BNNs) and their potential applications in synthetic biological intelligence and neurocomputing. Based on the context, it can be inferred that GenAI differs from traditional AI in its focus on detecting fine-grained activities in newborn resuscitation videos and the challenges in recognizing them, while BNNs (biological neurons) offer distinct benefits such as power efficiency, parallel dynamics, and robust functional connectivity that are not available to traditional AI systems. BNNs also require fewer resources and better align with human cognitive patterns through a proposed framework 

In [25]:
## Conversational example
print("Conversational RAG Example:")
chat_history = []

# First question
q1 = "What is machine learning?"
a1 = conversational_rag.invoke({
    "input": q1,
    "chat_history": chat_history
})

print(f"Q1: {q1}")
print(f"A1: {a1}")

Conversational RAG Example:
Q1: What is machine learning?
A1: Machine learning is the field of study focused on developing algorithms and models that allow computers to learn and make predictions or decisions based on data without being explicitly programmed to do so. It involves techniques such as deep learning, which uses neural networks with many layers to learn patterns and features from large datasets and can achieve human-level performance in tasks such as image and speech recognition, as well as natural language processing. In the context provided, three documents discuss different aspects of this field. Document 1 mentions the use of deterministic deep neural networks for medical image analysis, which have demonstrated high accuracy but can suffer from overconfidence in their predictions, making it difficult for healthcare professionals to determine when to trust the model's output. Document 2 discusses the need for predicting aggregate statistics for a population based on data

In [26]:
# Update history
chat_history.extend([
    HumanMessage(content=q1),
    AIMessage(content=a1)
])

In [27]:
# Follow-up question
q2 = "How is it different from traditional programming?"
a2 = conversational_rag.invoke({
    "input": q2,
    "chat_history": chat_history
})
print(f"\nQ2: {q2}")
print(f"A2: {a2}")


Q2: How is it different from traditional programming?
A2: The provided context doesn't directly compare machine learning to traditional programming, but it mentions the potential for large language models (LLMs) to serve as interactive curriculum designers for embodied AI systems and the need for understanding uncertainty and confidence in machine learning models, as discussed in Document 1 and 2. However, in Document 3, it introduces a new approach called Spatially-Aware Generation (SAGen) that combines a Forward model (FM) with an MLLM-based planner for precise constraint enforcement and explicit coordinate specification for layout planning, offering advantages such as efficiency and strong reliability in complex spatial prompts, which differs from traditional programming in that it is a model-based approach for generating layout plans instead of traditional programming that follows explicit instructions.
